### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [3]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,1000]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.3502044985863347  /  0.016882341997579584
estimators:  10  prof:  5  | R2 mean / std:  0.4197182529926469  /  0.032559126040067284
estimators:  10  prof:  10  | R2 mean / std:  0.3862264913707412  /  0.023428413803257275
estimators:  100  prof:  1  | R2 mean / std:  0.3322910328148504  /  0.03404655779236188
estimators:  100  prof:  5  | R2 mean / std:  0.42629546721514533  /  0.034997651912005295
estimators:  100  prof:  10  | R2 mean / std:  0.42737615503239595  /  0.035434549330450295
estimators:  1000  prof:  1  | R2 mean / std:  0.3357152476116722  /  0.03640265449537399
estimators:  1000  prof:  5  | R2 mean / std:  0.4301173254803366  /  0.03613580695674186
estimators:  1000  prof:  10  | R2 mean / std:  0.42297283296025306  /  0.03825792367336403


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [5]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [9]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [10]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [11]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [12]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [13]:
grid.best_score_

0.4225005250722306

In [14]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [15]:
grid.cv_results_

{'mean_fit_time': array([0.02327029, 1.33078766, 0.01695267, 1.69912283]),
 'std_fit_time': array([1.52873314e-02, 2.50454940e-02, 4.70973091e-06, 2.36371525e-02]),
 'mean_score_time': array([0.00165272, 0.0704778 , 0.00131631, 0.09308179]),
 'std_score_time': array([0.00046619, 0.00417922, 0.0004503 , 0.0098306 ]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.32474531, 0.35100446, 0.3206056 , 0.37089418]),
 'split1_test_score': array([0.44905111, 0.44919614, 0.36595797, 0.44955746]),
 'split2_test_score': array(

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [16]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import time

### Preprocessamento dos dados

#### Valores nulos

In [18]:
print(df.isnull().sum())

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64


In [19]:
print('Percentual de nulos:', str(round((df.total_quartos.isnull().sum()/df.size)*100,4))+'%')

Percentual de nulos: 0.1003%


In [20]:
df = df.dropna()
df.isnull().sum()

longitude                  0
latitude                   0
idade_mediana_das_casas    0
total_comodos              0
total_quartos              0
populacao                  0
familias                   0
salario_mediano            0
preco_mediano_das_casas    0
proximidade_ao_mar         0
dtype: int64

#### Variáveis categóricas

In [21]:
def separacao_variaveis(df):
    
    variavel_categorica = []
    variavel_numerica = []
    
    for variavel in df.columns:
        try:
            df[variavel].sum() / 2 #Se não resultar em um erro, a coluna será do tipo numérica
            variavel_numerica.append(variavel)
            
        except:
            variavel_categorica.append(variavel) #Em caso de erro, será do tipo categórica
    return variavel_categorica, variavel_numerica


def tratamento_variaveis(df,
                         variavel_categorica,
                         variavel_numerica,
                         variavel_target,
                         dataset_de_treino = True,
                         cat_encoder = None, 
                         std_scaler = None, 
                         ):
    
    if dataset_de_treino:  
        
        #OHE
        encoder = OneHotEncoder()
        df_variavel_categorica = encoder.fit_transform(df[variavel_categorica]).toarray()

        #Normalização
        sc = StandardScaler()
        df_variavel_numerica = sc.fit_transform(df[variavel_numerica])
        
        X, y =  np.c_[df_variavel_categorica, df_variavel_numerica], variavel_target.values

        return X, y, encoder, sc
    
    else:
        
        #OHE
        df_variavel_categorica = cat_encoder.transform(df[variavel_categorica]).toarray()
        
        #normalização
        df_variavel_numerica = std_scaler.transform(df[variavel_numerica]) 
        
        X, y =  np.c_[df_variavel_categorica, df_variavel_numerica], variavel_target.values
        
        return X, y

In [22]:
variavel_categorica, variavel_numerica = separacao_variaveis(df=df)
print(variavel_categorica)
print()
print(variavel_numerica)

['proximidade_ao_mar']

['longitude', 'latitude', 'idade_mediana_das_casas', 'total_comodos', 'total_quartos', 'populacao', 'familias', 'salario_mediano', 'preco_mediano_das_casas']


In [23]:
dftrain, dftest = train_test_split(df, test_size = 0.25, random_state = 0)

In [24]:
Xtrain, ytrain, encoder_train, std_scaler = tratamento_variaveis(df = dftrain,
                                                               variavel_categorica = variavel_categorica,
                                                               variavel_numerica = variavel_numerica,
                                                               variavel_target = dftrain.preco_mediano_das_casas,
                                                               dataset_de_treino = True,
                                                               cat_encoder = None, 
                                                               std_scaler = None, 
                                                              )
Xtrain.shape, ytrain.shape, dftrain.shape

((15324, 14), (15324,), (15324, 10))

In [25]:
Xtest, ytest = tratamento_variaveis(df = dftest,
                                    variavel_categorica = variavel_categorica,
                                    variavel_numerica = variavel_numerica,
                                    variavel_target = dftest.preco_mediano_das_casas,
                                    dataset_de_treino = False,
                                    cat_encoder = encoder_train, 
                                    std_scaler = std_scaler, 
                                   )
Xtest.shape, ytest.shape, dftest.shape

((5109, 14), (5109,), (5109, 10))

#### Grid Search para os modelos

In [26]:
def grid(X,y,estimador_base,params,metrica,cv):
    
    grid = GridSearchCV(estimator = estimador_base,
                        param_grid = params, 
                        scoring = metrica, 
                        cv = cv
                       )
    
    grid.fit(X,y)
    
    best_params = grid.best_params_
    
    best_score = grid_knn.best_score_
    
    cv_results = grid.cv_results_
    
    return best_params, best_score, cv_results

### KNN Regressor

#### Dados de treino

In [130]:
t0 = time.time()

best_params_knn, best_score_knn, cv_results_knn = grid(X = Xtrain, 
                                           y = ytrain,
                                           estimador_base = KNeighborsRegressor(),
                                           params = [{'n_neighbors': [2,5,10], 'weights': ['uniform','distance'],'p':[1,2,5]}],
                                           metrica = 'r2',
                                           cv = 3)

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

tempo (em segundos) para execução:  40.12


In [131]:
best_params_knn

{'n_neighbors': 5, 'p': 5, 'weights': 'distance'}

In [132]:
best_score_knn

0.9732959583910256

### Random Forest

In [137]:
t0 = time.time()

best_params_rf, best_score_rf, cv_results_rf = grid(X = Xtrain, 
                                                    y = ytrain,
                                                    estimador_base = RandomForestRegressor(),
                                                    params = [{"n_estimators":[10,100,1000], 
                                                               "max_depth":[2,5,10]}],
                                                    metrica = 'r2',
                                                    cv = 3)

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

tempo (em segundos) para execução:  284.68


In [140]:
best_params_rf

{'max_depth': 10, 'n_estimators': 100}

In [141]:
best_score_rf

0.9732959583910256

### GradientBoostings

#### Dados de teste

In [173]:
t0 = time.time()

best_params_gbr, best_score_gbr, cv_results_gbr = grid(X = Xtrain, 
                                                       y = ytrain,
                                                       estimador_base = GradientBoostingRegressor(), 
                                                       params = [{"n_estimators":[10,100], 
                                                                  "max_depth":[2,10],
                                                                  "learning_rate": [0.1,0.01],
                                                                  "min_samples_split": [2,10]}],
                                                       metrica = 'r2',
                                                       cv = 3)

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

tempo (em segundos) para execução:  128.48


In [174]:
best_params_gbr

{'learning_rate': 0.1,
 'max_depth': 10,
 'min_samples_split': 10,
 'n_estimators': 100}

In [175]:
best_score_gbr

0.9732959583910256

### Todos os modelos tiveram uma assertividade impressionante nos dados com mais de 97% de acurácia.

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [203]:
class gridSearchAll():
    
    def __init__(self,num_folds):
        self.grid_models = []
        self.scoring = 'r2'
        self.num_folds = num_folds
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
        
    def fit_all(self,X,y):
        self.X = X
        self.y = y
        self.best_scores = {}
        
        for estimador, parametros in self.grid_models:
            
            grid = GridSearchCV(estimator = estimador,
                        param_grid = parametros, 
                        scoring = self.scoring, 
                        cv = self.num_folds
                               )
    
            grid.fit(X,y)
            print('Modelo', str(estimador).replace('()',''),'treinado')
            
            self.best_scores.update({str(estimador).replace('()',''):[grid.best_score_,grid.best_params_]})
        
        return self.best_scores
    
    def best_all_grid_models(self, best_scores):
        x = max(best_scores[i] for i in best_scores)
        print('A melhor modelo foi o', max(best_score, key=best_score.get), 'com',str(x[0]*100)+'% de acurácia,\n tendo como parâmetros',x[1])

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [204]:
gd = gridSearchAll(num_folds = 3)

gd.grid_models

[]

In [169]:
#Parâmetros Random Forest

params_RF = [{"n_estimators":[10,1000], 
              "max_depth":[2,10]}]

gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

gd.grid_models

[[RandomForestRegressor(),
  [{'n_estimators': [10, 1000], 'max_depth': [2, 10]}]]]

In [170]:
# Parâmetros KNN Regressor

paramsKNN = [{'n_neighbors': [2,10],
              'weights': ['uniform','distance'],
              'p':[1,5]}]

gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = paramsKNN)

gd.grid_models

[[RandomForestRegressor(),
  [{'n_estimators': [10, 1000], 'max_depth': [2, 10]}]],
 [KNeighborsRegressor(),
  [{'n_neighbors': [2, 10], 'weights': ['uniform', 'distance'], 'p': [1, 5]}]]]

In [171]:
# Parâmetros GradientBoostings

paramsGB = [{"n_estimators":[10,100], 
           "max_depth":[2,10],
           "learning_rate": [0.1,0.01],
           "min_samples_split": [2,10]}]

gd.insert_model(estimator_base = GradientBoostingRegressor(), param_grid = paramsGB)

gd.grid_models

[[RandomForestRegressor(),
  [{'n_estimators': [10, 1000], 'max_depth': [2, 10]}]],
 [KNeighborsRegressor(),
  [{'n_neighbors': [2, 10], 'weights': ['uniform', 'distance'], 'p': [1, 5]}]],
 [GradientBoostingRegressor(),
  [{'n_estimators': [10, 100],
    'max_depth': [2, 10],
    'learning_rate': [0.1, 0.01],
    'min_samples_split': [2, 10]}]]]

In [172]:
t0 = time.time()

best_score = gd.fit_all(Xtrain,ytrain)

t1 = time.time()
print("tempo (em segundos) para execução: ", np.round(t1-t0,2))

Modelo RandomForestRegressor treinado
Modelo KNeighborsRegressor treinado
Modelo GradientBoostingRegressor treinado
tempo (em segundos) para execução:  381.2


In [187]:
best_score

{'RandomForestRegressor': [0.9999980238935642,
  {'max_depth': 10, 'n_estimators': 1000}],
 'KNeighborsRegressor': [0.9714558927049305,
  {'n_neighbors': 10, 'p': 5, 'weights': 'distance'}],
 'GradientBoostingRegressor': [0.9999995722571619,
  {'learning_rate': 0.1,
   'max_depth': 10,
   'min_samples_split': 10,
   'n_estimators': 100}]}

In [205]:
gd.best_all_grid_models(best_score)

A melhor modelo foi o GradientBoostingRegressor com 99.99995722571619% de acurácia,
 tendo como parâmetros {'learning_rate': 0.1, 'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
